In [1]:
import os
import pandas as pd
import ast

In [2]:
# Colab warns and provides remediation steps if the GPUs is not compatible with RAPIDS.

!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 553, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 553 (delta 179), reused 147 (delta 100), pack-reused 269 (from 1)
Receiving objects: 100% (553/553), 178.44 KiB | 2.38 MiB/s, done.
Resolving deltas: 100% (281/281), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.9 MB/s eta 0:00:00
Installing RAPIDS remaining 24.10.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.7/567.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.5/915.5 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
def file_load(filepath):

    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    'category', categories=SUBSETS, ordered=True)
        except (ValueError, TypeError):
            # the categories and ordered arguments were removed in pandas 0.25
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks

In [4]:
tracks = file_load('fma_metadata/tracks.csv')
genres = file_load('fma_metadata/genres.csv')
features = file_load('fma_metadata/features.csv')


In [5]:
tracks = tracks.sample(n=30000, random_state=42)

In [6]:
track_ids = tracks.index.unique()
features = features.loc[track_ids]


In [7]:
best_statistics = {
    'mfcc': ['mean', 'std'],
    'chroma_stft': ['mean', 'max'],
    'chroma_cens': ['mean', 'max'],
    'chroma_cqt': ['mean'],
    'spectral_centroid': ['mean', 'std'],
    'spectral_bandwidth': ['mean', 'max'],
    'spectral_rolloff': ['mean'],
    'spectral_contrast': ['mean'],
    'rms': ['mean'],
    'zcr': ['mean', 'std'],
    'tonnetz': ['mean']
}
selected_columns = []
for feature, stats in best_statistics.items():
    for stat in stats:
        # Filter multi-index tuples that match the feature and stat
        matched_cols = [col for col in features.columns if col[0] == feature and col[1] == stat]
        selected_columns.extend(matched_cols)
features = features[selected_columns]


In [8]:
features = features.loc[:, features.columns.get_level_values('number').isin(['01'])]
features.head()

feature           mfcc            chroma_stft       ... spectral_contrast       zcr             tonnetz
statistics        mean        std        mean  max  ...              mean      mean       std      mean
number              01         01          01   01  ...                01        01        01        01
track_id                                            ...                                                
43742      -271.812408  96.714348    0.272961  1.0  ...         16.684275  0.038113  0.051893  0.017989
31941       -94.052002  55.837280    0.483759  1.0  ...         17.800627  0.027171  0.019550 -0.020757
50985      -452.438446  73.048615    0.338060  1.0  ...         32.151752  0.020377  0.009784 -0.008503
146981     -478.325043  82.608185    0.220894  1.0  ...         29.622980  0.093110  0.043997 -0.005130
20461      -121.869453  98.762352    0.439927  1.0  ...         18.266766  0.050857  0.040272 -0.000254

[5 rows x 16 columns]

In [9]:
flattened_features_columns = [('feature', '_'.join(filter(None, col))) for col in features.columns]
features.columns = pd.MultiIndex.from_tuples(flattened_features_columns)
df = pd.merge(features, tracks, left_index=True, right_index=True)

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import cuml
from cuml.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import tensorflow as tf

def get_model(model_name, **kwargs):
    """
    Returns a machine learning or deep learning model based on the given model name.

    Parameters:
    - model_name (str): Name of the model (case insensitive). Valid options are:
      - 'logistic regression'
      - 'decision tree'
      - 'random forest'
      - 'k-nearest neighbors'
      - 'support vector machine'
      - 'naive bayes'
      - 'neural network'
      - 'convolutional neural network'
    - kwargs: Additional parameters to pass to the model constructor.

    Returns:
    - A machine learning or deep learning model.
    """
    model_name = model_name.lower()

    if model_name == 'logistic regression':
        return LogisticRegression(**kwargs)
    elif model_name == 'decision tree':
        return DecisionTreeClassifier(**kwargs)
    elif model_name == 'random forest':
        return RandomForestClassifier(**kwargs)
    elif model_name == 'k-nearest neighbors':
        return KNeighborsClassifier(**kwargs)
    elif model_name == 'support vector machine':
        return LinearSVC(**kwargs)
    elif model_name == 'naive bayes':
        return GaussianNB(**kwargs)
    elif model_name == 'neural network':
        return MLPClassifier(**kwargs)
    else:
        raise ValueError(f"Invalid model name '{model_name}'. Please choose from: "
                         "'logistic regression', 'decision tree', 'random forest', 'k-nearest neighbors', "
                         "'support vector machine', 'naive bayes', 'neural network', or 'convolutional neural network'.")

In [100]:
X = df.xs('feature', axis=1, level=0)
y = df[('track', 'genres')].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

In [101]:
y.isnull().sum()

610

In [102]:
combined_df = X.copy()
combined_df['target'] = y
cleaned_df = combined_df.dropna()
X = cleaned_df.drop(columns=['target'])
y = cleaned_df['target']

In [103]:
y.isnull().sum()

0

In [104]:
y = y.astype(int).astype('category').cat.codes

In [105]:
counts = y.value_counts()

valid_classes = counts[counts >= 10].index
top_20_classes = counts.index[:20]

# Filter X and y to include only these valid classes
X = X[y.isin(top_20_classes)]
y = y[y.isin(top_20_classes)]

In [106]:
y

,0
track_id,
43742,2
31941,44
50985,3
146981,0
20461,20
...,...
85240,11
5194,9
51763,9


## Genres prediction

In [107]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

In [78]:
X_train.dtypes

,0
mfcc_mean_01,float64
mfcc_std_01,float64
chroma_stft_mean_01,float64
chroma_stft_max_01,float64
chroma_cens_mean_01,float64
chroma_cens_max_01,float64
chroma_cqt_mean_01,float64
spectral_centroid_mean_01,float64
spectral_centroid_std_01,float64
spectral_bandwidth_mean_01,float64


In [108]:


# Check that X is a Pandas DataFrame
print(type(X))  # Should be <class 'pandas.core.frame.DataFrame'>

X_cudf = cudf.from_pandas(X)
print(type(X_cudf))  # Should be <class 'cudf.core.dataframe.DataFrame'>

<class 'pandas.core.frame.DataFrame'>
<class 'cudf.core.dataframe.DataFrame'>


In [109]:
from IPython.display import display
from sklearn.metrics import classification_report
models = {
    'logistic_regression': get_model('logistic regression', max_iter=1000),
    'decision_tree': get_model('decision tree', max_depth=10),
    'random_forest': get_model('random forest', n_estimators=100),
    'naive_bayes': get_model('naive bayes'),
    'k_nearest_neighbors': get_model('k-nearest neighbors', n_neighbors=5),
    'support_vector_machine': get_model('support vector machine'),
    'neural_network': get_model('neural network', hidden_layer_sizes=(200, 100)),
}

accuracies = {}

for model_name, model in models.items():
    if model_name == 'support_vector_machine':
    # Train the model
      model.fit(X_train.values, y_train)
    else:
      model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    print(f'\n\nClassification Report for {model_name}:')
    display(report)

    acc = model.score(X_test, y_test)
    accuracies[model_name] = acc

# Convert accuracies dict to DataFrame
accuracy_df = pd.DataFrame.from_dict(accuracies, orient='index', columns=['accuracy'])

print("\n\nAccuracy Table:")
display(accuracy_df)



Classification Report for logistic_regression:


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre

{'0': {'precision': 0.21927710843373494,
  'recall': 0.19077568134171907,
  'f1-score': 0.2040358744394619,
  'support': 477.0},
 '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 99.0},
 '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 92.0},
 '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 102.0},
 '4': {'precision': 0.21875,
  'recall': 0.14432989690721648,
  'f1-score': 0.17391304347826086,
  'support': 97.0},
 '9': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 330.0},
 '11': {'precision': 0.21052631578947367,
  'recall': 0.23404255319148937,
  'f1-score': 0.2216624685138539,
  'support': 376.0},
 '14': {'precision': 0.289426523297491,
  'recall': 0.8485113835376532,
  'f1-score': 0.4316258351893096,
  'support': 1142.0},
 '16': {'precision': 0.06944444444444445,
  'recall': 0.02092050209205021,
  'f1-score': 0.03215434083601286,
  'support': 239.0},
 '17': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'supp



Classification Report for decision_tree:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...



Classification Report for neural_network:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'0': {'precision': 0.07692307692307693,
  'recall': 0.0020964360587002098,
  'f1-score': 0.004081632653061225,
  'support': 477.0},
 '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 99.0},
 '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 92.0},
 '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 102.0},
 '4': {'precision': 0.19689119170984457,
  'recall': 0.3917525773195876,
  'f1-score': 0.2620689655172414,
  'support': 97.0},
 '9': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 330.0},
 '11': {'precision': 0.22401433691756273,
  'recall': 0.3324468085106383,
  'f1-score': 0.2676659528907923,
  'support': 376.0},
 '14': {'precision': 0.2844950213371266,
  'recall': 0.8756567425569177,
  'f1-score': 0.42946102641185313,
  'support': 1142.0},
 '16': {'precision': 0.09259259259259259,
  'recall': 0.10460251046025104,
  'f1-score': 0.09823182711198428,
  'support': 239.0},
 '17': {'precision': 0.1016949152542373,
  'reca



Accuracy Table:


,accuracy
logistic_regression,0.262598
decision_tree,0.258984
random_forest,0.310440
naive_bayes,0.200936
k_nearest_neighbors,0.211354
support_vector_machine,0.254731
neural_network,0.257495


In [110]:
X.shape

(23512, 16)

## Year of release Prediction

In [32]:
df[('album','date_released')]

,album
,date_released
track_id,
43742,NaT
31941,2009-02-12
50985,NaT
146981,2016-11-18
20461,NaT
...,...
85240,2013-06-01
5194,2004-09-13


In [117]:
# Assuming 'date_released' is the column containing the datetime data
df[('album','year')] = df[('album', 'date_released')].dt.year

In [118]:
X = df.xs('feature', axis=1, level=0)
y = df[('album','year')]

In [119]:
combined_df = X.copy()
combined_df['target'] = y
cleaned_df = combined_df.dropna()
X = cleaned_df.drop(columns=['target'])
y = cleaned_df['target']

In [120]:
counts = y.value_counts()

valid_classes = counts[counts >= 10].index
top_20_classes = counts.index[:20]

# Filter X and y to include only these valid classes
X = X[y.isin(valid_classes)]
y = y[y.isin(valid_classes)]

In [121]:
y.value_counts()

,count
target,
2013.0,2219
2012.0,2194
2011.0,2167
2010.0,1924
2014.0,1795
2015.0,1784
2009.0,1738
2016.0,1482
2008.0,1181


In [122]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

In [123]:
from sklearn.metrics import classification_report
models = {
    'logistic_regression': get_model('logistic regression', max_iter=1000),
    'decision_tree': get_model('decision tree', max_depth=10),
    'random_forest': get_model('random forest', n_estimators=100),
    'naive_bayes': get_model('naive bayes'),
    'k_nearest_neighbors': get_model('k-nearest neighbors', n_neighbors=5),
    'support_vector_machine': get_model('support vector machine'),
    'neural_network': get_model('neural network', hidden_layer_sizes=(200, 100)),
}

accuracies = {}

for model_name, model in models.items():
    if model_name == 'support_vector_machine':
    # Train the model
      model.fit(X_train.values, y_train)
    else:
      model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    print(f'\n\nClassification Report for {model_name}:')
    display(report)

    acc = model.score(X_test, y_test)
    accuracies[model_name] = acc

# Convert accuracies dict to DataFrame
accuracy_df = pd.DataFrame.from_dict(accuracies, orient='index', columns=['accuracy'])

print("\n\nAccuracy Table:")
display(accuracy_df)



Classification Report for logistic_regression:


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre

{'1982.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4.0},
 '1993.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7.0},
 '1995.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9.0},
 '1996.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0},
 '1997.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0},
 '1998.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 15.0},
 '1999.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 22.0},
 '2000.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 13.0},
 '2001.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 19.0},
 '2002.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 24.0},
 '2003.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 50.0},
 '2004.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 73.0},
 '2005.0': {'precision': 0.0, 'recall': 0.0



Classification Report for decision_tree:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...



Classification Report for neural_network:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'1982.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4.0},
 '1993.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7.0},
 '1995.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9.0},
 '1996.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0},
 '1997.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0},
 '1998.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 15.0},
 '1999.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 22.0},
 '2000.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 13.0},
 '2001.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 19.0},
 '2002.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 24.0},
 '2003.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 50.0},
 '2004.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 73.0},
 '2005.0': {'precision': 0.0, 'recall': 0.0



Accuracy Table:


,accuracy
logistic_regression,0.113188
decision_tree,0.121021
random_forest,0.144265
naive_bayes,0.117736
k_nearest_neighbors,0.099545
support_vector_machine,0.120010
neural_network,0.111925
